In [2]:
import numpy as np
import matplotlib.pyplot as plt
import portaudio

ModuleNotFoundError: No module named 'portaudio'

## Record Audio

In [ ]:
FORMAT = pyaudio.paInt16
SAMPLEFREQ = 44100
FRAMESIZE = 1024
NOFFRAMES = 220
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT, channels=1, rate=SAMPLEFREQ, input=True, frames_per_buffer=FRAMESIZE)
data = stream.read(NOFFRAMES*FRAMESIZE)
decoded = np.frombuffer(data, "int16")

stream.stop_stream()
stream.close()
plt.plot(decoded)
plt.show()

In [4]:
import pyaudio
import numpy
import matplotlib.pyplot as plt
FORMAT = pyaudio.paInt16
SAMPLEFREQ = 44100
FRAMESIZE = 1024
NOFFRAMES = 220
p = pyaudio.PyAudio()
print('running')
stream = p.open(format=FORMAT,channels=1,rate=SAMPLEFREQ,
                input=True,frames_per_buffer=FRAMESIZE)
data = stream.read(NOFFRAMES*FRAMESIZE)
decoded = numpy.fromstring(data, 'Int16')
stream.stop_stream()
stream.close()
p.terminate()
print('done')
plt.plot(decoded)
plt.show()

Could not import the PyAudio C module 'pyaudio._portaudio'.


ImportError: dlopen(/Users/hannesbrugger/opt/anaconda3/lib/python3.9/site-packages/pyaudio/_portaudio.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '_PaMacCore_SetupChannelMap'